# Simple RNN for bit parity problem
Let's create a RNN that looks at a series of characters and reverses it

In [1]:
import random
import string
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
char_id_dict = {char: idx for idx, char in enumerate(string.ascii_lowercase)}
id_char_dict = {idx: char for idx, char in enumerate(string.ascii_lowercase)}

def char2id(strArray):
    return map(lambda c:char_id_dict[c], strArray)

def id2char(intArray):
    return map(lambda c:id_char_dict[c], intArray)

print(char_id_dict)
print(id_char_dict)

{'t': 19, 'g': 6, 'c': 2, 'z': 25, 'l': 11, 'k': 10, 'h': 7, 'p': 15, 'a': 0, 'b': 1, 'i': 8, 'y': 24, 'w': 22, 'u': 20, 'e': 4, 'f': 5, 'o': 14, 'j': 9, 'n': 13, 'q': 16, 's': 18, 'm': 12, 'r': 17, 'v': 21, 'd': 3, 'x': 23}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


In [149]:
def random_chars(length):
    chars = ''.join(random.choice(string.ascii_lowercase) for _ in range(length))
    chars = list(chars)
    chars = list(char2id(chars))
    return chars, chars[::-1]

s, rev_s = random_chars(5)

print(s)
print(rev_s)

one_hot = list(id2char(s))
print(one_hot)
print(list(char2id(one_hot)))

[21, 3, 9, 10, 19]
[19, 10, 9, 3, 21]
['v', 'd', 'j', 'k', 't']
[21, 3, 9, 10, 19]


In [150]:
def random_string_generator(batch_size=128, chars=5):
    while True:
        xy = [random_chars(chars) for _ in range(batch_size)]
        x, y = zip(*((pair[0], pair[1]) for pair in xy))
        yield np.array(x).reshape(batch_size, -1), np.array(y).reshape(batch_size, -1)

randomStringGenerator = random_string_generator(batch_size=3)

x, y = next(randomStringGenerator)
print(x.shape, y.shape)
print(x)
print(y)

(3, 5) (3, 5)
[[18 21 10 21 25]
 [ 1 20 17  8 16]
 [25  5 12  7  5]]
[[25 21 10 21 18]
 [16  8 17 20  1]
 [ 5  7 12  5 25]]


In [7]:
tf_x = tf.placeholder(tf.float32, shape=[None, 128, 5])
tf_y = tf.placeholder(tf.int32, shape=[None, 128, 5])

tf_init_state = tf.placeholder(tf.float32, [None, 5])


iw = tf.Variable(tf.random_normal(shape=[5, 5]), dtype=tf.float32)
ib = tf.Variable(tf.random_normal(shape=[1, 5]), dtype=tf.float32)

sw = tf.Variable(tf.random_normal(shape=[5, 5]), dtype=tf.float32)
sb = tf.Variable(tf.random_normal(shape=[1, 5]), dtype=tf.float32)




In [8]:
def print_func(x):
    print(x)
    return x


def fn(previous_output, current_input):
#     abc = tf.py_func(print_func, [previous_output], [tf.float32])
    return previous_output * current_input

elems = tf.Variable([1.0, 2.0, 2.0, 2.0])
initializer = tf.constant(1.0)
out = tf.scan(fn, elems, initializer=initializer)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(out))
#     print(prev[1].eval())
    


[ 1.  2.  4.  8.]


# Simple RNN to estimate cumulative sum

In [ ]:
def input_target_generator(min_duration=5, max_duration=10):
    """ Generate toy input, target sequences.
    
    Each input sequence has values that are drawn from the standard normal
    distribution, and each target sequence is the corresponding cumulative sum.
    Sequence durations are chosen at random using a discrete uniform
    distribution over `[min_duration, max_duration]`.
    
    Args:
        min_duration: A positive integer. The minimum sequence duration.
        max_duration: A positive integer. The maximum sequence duration.

    Yields:
        A tuple,
        inputs: A 2-D float32 NumPy array with shape `[duration, 1]`.
        targets: A 2-D float32 NumPy array with shape `[duration, 1]`.
    """
    
    while True:
        duration = np.random.randint(min_duration, max_duration)
        inputs = np.random.randn(duration).astype(np.float32)
        inputs = np.random.randint(10, size=5)
        targets = np.cumsum(inputs).astype(np.float32)
        yield inputs.reshape(-1, 1), targets.reshape(-1, 1)
        
gen = input_target_generator()

x, y = next(gen)
print(x)
print(y)

In [ ]:
# http://blog.gaurav.im/2017/01/11/a-gentle-intro-to-recurrent-nns-in-tensorflow/
tf.reset_default_graph()

input_size = 1
# no_steps = 5
hidden_size = 256
x = tf.placeholder(tf.float32, shape=[None, input_size], name="x")
y = tf.placeholder(tf.float32, shape=[None, input_size], name="y")
init_state = tf.placeholder(tf.float32, shape=[hidden_size, 1], name='state')

def rnn_cell(x_input, h_prev):
    with tf.variable_scope('rnn_cell') as scope:
        # seems the best way to bypass the reuse=True chicken egg scenario
        try:
            tf.get_variable('test', [1])
        except ValueError:
            scope.reuse_variables()
        #init variables
        Wxh = tf.get_variable('Wxh', [hidden_size, input_size])
        Whh = tf.get_variable('Whh', [hidden_size, hidden_size])
        Why = tf.get_variable('Why', [input_size, hidden_size])
        bh = tf.get_variable('bh', [hidden_size, 1])
        by = tf.get_variable('by', [input_size, 1])
        scope.reuse_variables()
    
    x_input = tf.expand_dims(x_input, 0)
    # (hidden,input) * (input,input)  + (hidden,hidden) * (hidden,1) + (hidden, 1)
    # = (hidden,input) + (hidden, 1) + (hidden,1)
    # = (hidden, input)
    next_state = tf.tanh(tf.matmul(Wxh, x_input) + tf.matmul(Whh, h_prev) + bh)
    
    # (input,hidden) * (hidden, 1) + (input, 1)
    y_output = tf.matmul(Why, next_state) + by
    return y_output, next_state

rnn_inputs = tf.unstack(x, axis=1)
rnn_targets = tf.unstack(y, axis=1)
rnn_outputs = []

state = init_state
for rnn_input in rnn_inputs:
    y_out, state = rnn_cell(rnn_input, state)
    y_out = tf.squeeze(y_out)
    rnn_outputs.append(y_out)

gen = input_target_generator()

# this is for testing a single run
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sample_x, sample_y = next(gen)
#     feed_dict = {x: sample_x, y: sample_y}
#     i, o, t, l = sess.run([rnn_inputs, rnn_outputs, rnn_targets, loss], feed_dict=feed_dict)
#     print('i', i)
#     print('o', o)
#     print('t', t)
#     print('l', l)

total_loss = tf.squared_difference(rnn_targets, rnn_outputs, name="total_loss")
# total_loss = tf.square(tf.subtract(rnn_targets, rnn_outputs))
loss = tf.reduce_sum(total_loss, name='mean_loss')  #mean of all losses

#learning_rate = 10e-8
global_step = tf.Variable(0, trainable=False, dtype=tf.int32)
learning_rate = tf.train.exponential_decay(1e-2, global_step, 100, 0.9, staircase=True)
# learning_rate = 1e-2
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

# tf.scalar_summary('loss', loss)

# Optimizer.
# trainables = tf.trainable_variables()
# grads = tf.gradients(loss, trainables)
# grads, _ = tf.clip_by_global_norm(grads, clip_norm=1.0)
# grad_var_pairs = zip(grads, trainables)

# global_step = tf.Variable(0, trainable=False, dtype=tf.int32)
# learning_rate = tf.train.exponential_decay(1e-2, global_step, 15000, 0.1, staircase=True)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# optimizer = optimizer.apply_gradients(grad_var_pairs,global_step=global_step)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(100000):
        sample_x, sample_y = next(gen)
        sample_x = np.array([[1],[2], [3], [4]])
        sample_y = np.array([[1], [3], [6], [10]])
        epoch_state = np.zeros([hidden_size, 1])
        feed_dict = {x: sample_x, y: sample_y, init_state: epoch_state}
        
        o, l, opt = sess.run([rnn_outputs, loss, optimizer], feed_dict=feed_dict)
#         print('run {0} --------'.format(epoch))
#         print('target:', sample_y.squeeze())
        print('target:', np.squeeze(sample_y))
        print('output:', o)
        print('loss:', l)

        
    sample_x = np.array([[1],[2], [2], [2]])
    sample_y = np.array([[1], [3], [5], [7]])
    epoch_state = np.zeros([hidden_size, 1])
    feed_dict = {x: sample_x, y: sample_y, init_state: epoch_state}

    o, l, opt = sess.run([rnn_outputs, loss, optimizer], feed_dict=feed_dict)
    print('test')
    print('output:', o)
    print('loss:', l)

# Let's try chars instead, numbers didn't work very well

In [ ]:
randomStringGenerator = random_string_generator(batch_size=1)
x, y = next(randomStringGenerator)
print(x.shape, y.shape)
print(x)
print(y)

sess = tf.InteractiveSession()
tf_x = tf.constant(x)
tf_y = tf.constant(y)

tf_x_oh = tf.one_hot(tf_x, depth=26)
tf_y_oh = tf.one_hot(tf_y, depth=26)

test_x_oh = tf_x_oh.eval()
print(list(id2char(x.squeeze().tolist())))
print(test_x_oh.shape)
print(np.argmax(test_x_oh, axis=2))

sess.close()


In [ ]:
tf.reset_default_graph()


sequence_length=3
input_size = 26 #no. of chars in onehot
hidden_size = 128 #no. of hidden units

tf_x = tf.placeholder(tf.int32, shape=[sequence_length], name="x")
tf_y = tf.placeholder(tf.int32, shape=[sequence_length], name="y")
tf_x_oh = tf.cast(tf.one_hot(tf_x, depth=26), tf.float32)
tf_y_oh = tf.cast(tf.one_hot(tf_y, depth=26), tf.float32)

init_state = tf.placeholder(tf.float32, shape=[hidden_size, 1], name='state')

# out = sess.run([tf_x_oh], feed_dict={tf_x: x})
# print(np.array(out).shape)

with tf.variable_scope('rnn_cell') as scope:
    #init variables
    Wxh = tf.get_variable('Wxh', [hidden_size, input_size])
    Whh = tf.get_variable('Whh', [hidden_size, hidden_size])
    Why = tf.get_variable('Why', [input_size, hidden_size])
    bh = tf.get_variable('bh', [hidden_size, 1])
    by = tf.get_variable('by', [input_size, 1])


def rnn_cell(x_input, h_prev):
    with tf.variable_scope('rnn_cell', reuse=True) as scope:
        Wxh = tf.get_variable('Wxh', [hidden_size, input_size])
        Whh = tf.get_variable('Whh', [hidden_size, hidden_size])
        Why = tf.get_variable('Why', [input_size, hidden_size])
        bh = tf.get_variable('bh', [hidden_size, 1])
        by = tf.get_variable('by', [input_size, 1])
    
    x_input = tf.expand_dims(x_input, 1)
    # (hidden,input) * (input,input)  + (hidden,hidden) * (hidden,1) + (hidden, 1)
    # = (hidden,input) + (hidden, 1) + (hidden,1)
    # = (hidden, input)
    next_state = tf.tanh(tf.matmul(Wxh, x_input) + tf.matmul(Whh, h_prev) + bh)
    
    # (input,hidden) * (hidden, 1) + (input, 1)
    y_output = tf.matmul(Why, next_state) + by
    return y_output, next_state


rnn_inputs = tf.unstack(tf_x_oh)
rnn_targets = tf.unstack(tf_y_oh)
rnn_outputs = []

# out = sess.run(rnn_inputs, feed_dict={tf_x: x[0]})
# print(out)


state = init_state
for rnn_input in rnn_inputs:
    y_out, state = rnn_cell(rnn_input, state)
    y_out = tf.squeeze(y_out)
    rnn_outputs.append(y_out)

# sess.run(tf.global_variables_initializer())
# epoch_state = np.zeros([hidden_size, 1])
# out, target = sess.run([rnn_outputs, rnn_targets], feed_dict={tf_x: x[0], init_state: epoch_state, tf_y: y[0]})
# print(out[0].shape)

predictions = tf.argmax(rnn_outputs, axis=1)

losses = [tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target) 
              for logits, target in zip(rnn_outputs, rnn_targets)]
total_loss = tf.reduce_mean(losses)
learning_rate = 1e-1
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(20000):
        randomStringGenerator = random_string_generator(batch_size=256, chars=sequence_length)
        x, y = next(randomStringGenerator)
        print('run {0} --------'.format(epoch))
        
        for i in range(x.shape[0]):
#             print('sample {0}'.format(i))
            sample_x = x[i]
            sample_y = y[i]
            epoch_state = np.zeros([hidden_size, 1])
            feed_dict = {tf_x: sample_x, tf_y: sample_y, init_state: epoch_state}
            o, l, opt = sess.run([predictions, total_loss, optimizer], feed_dict=feed_dict)
    #         print('target:', sample_y.squeeze())
        print('target:', sample_y)
        print('output:', o)
        print('loss:', l)



# That also doesn't seem to work
# let's use Tensorflow's RNN cells

In [134]:
tf.reset_default_graph()

batch_size=4
sequence_length=2
input_size = 26 #no. of chars in onehot
hidden_size = 128 #no. of hidden units

tf_x = tf.placeholder(tf.int32, shape=[batch_size, sequence_length], name="x")
tf_y = tf.placeholder(tf.int32, shape=[batch_size, sequence_length], name="y")
tf_x_oh = tf.cast(tf.one_hot(tf_x, depth=input_size), tf.float32)
tf_y_oh = tf.cast(tf.one_hot(tf_y, depth=input_size), tf.float32)

rnn_outputs = []

cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
init_state = cell.zero_state(batch_size=batch_size, dtype=tf.float32)
rnn_outputs, state = tf.nn.dynamic_rnn(cell, inputs=tf_x_oh, initial_state=init_state, dtype=tf.float32)

Why = tf.Variable(tf.random_normal(shape=[batch_size, hidden_size, input_size], dtype=tf.float32))
by = tf.Variable(tf.random_normal(shape=[batch_size, 1, input_size], dtype=tf.float32))

logits = tf.matmul(rnn_outputs, Why) + by
predictions = tf.argmax(logits, axis=2)

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_y_oh)
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(40001):
        randomStringGenerator = random_string_generator(batch_size=batch_size, chars=sequence_length)
        x, y = next(randomStringGenerator)
#         print('run {0} --------'.format(epoch))

        feed_dict = {tf_x: x, tf_y: y}
        o, l, opt = sess.run([predictions, loss, optimizer], feed_dict=feed_dict)
        if(epoch % 1000 == 0):
            print('run {0} --------'.format(epoch))
#             print('input:', x)
            print('target:', y)
            print('output:', o)
            print('loss:', l)



run 0 --------
target: [[20 10]
 [ 2 12]
 [17 15]
 [11 12]]
output: [[20  3]
 [ 7  5]
 [23 17]
 [23 18]]
loss: [[ 0.75843024  6.73534966]
 [ 5.32160854  6.38236284]
 [ 2.81987858  4.33962154]
 [ 4.14901686  1.68291402]]
run 1000 --------
target: [[13 16]
 [15 21]
 [10  4]
 [24 25]]
output: [[ 1 16]
 [14 21]
 [14  4]
 [11 25]]
loss: [[ 3.25473118  0.03441959]
 [ 2.63751173  0.0356067 ]
 [ 3.23337984  0.00560557]
 [ 2.79000473  0.00688774]]
run 2000 --------
target: [[ 3 19]
 [20 25]
 [22 14]
 [ 9  6]]
output: [[ 2 19]
 [23 25]
 [23 14]
 [14  6]]
loss: [[ 3.05199981  0.01422621]
 [ 4.00644588  0.00576702]
 [ 2.93528938  0.01180861]
 [ 3.82049513  0.00808502]]
run 3000 --------
target: [[16 22]
 [ 4  6]
 [ 0 20]
 [14  1]]
output: [[16 22]
 [14  6]
 [20 20]
 [19  1]]
loss: [[ 2.47166443  0.01254242]
 [ 4.19073772  0.01824373]
 [ 3.41091251  0.00890333]
 [ 4.03273249  0.01146867]]
run 4000 --------
target: [[17 16]
 [25 15]
 [ 5 20]
 [24  6]]
output: [[10 16]
 [10 15]
 [15 20]
 [ 2  6]]
los

# That can do batch but still doesn't yield best results
# Let's try LSTM

In [229]:
tf.reset_default_graph()

batch_size=4
sequence_length=5
input_size = 26 #no. of chars in onehot
hidden_size = 256 #no. of hidden units

tf_x = tf.placeholder(tf.int32, shape=[batch_size, sequence_length], name="x")
tf_y = tf.placeholder(tf.int32, shape=[batch_size, sequence_length], name="y")
tf_x_oh = tf.cast(tf.one_hot(tf_x, depth=input_size), tf.float32) # [batch, sequence_length, 26]
tf_y_oh = tf.cast(tf.one_hot(tf_y, depth=input_size), tf.float32) # [batch, sequence_length, 26]

rnn_outputs = []

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
rnn_outputs, state = tf.nn.dynamic_rnn(cell, inputs=tf_x_oh, dtype=tf.float32)

Why = tf.Variable(tf.random_normal(shape=[batch_size, hidden_size, input_size], dtype=tf.float32))
by = tf.Variable(tf.random_normal(shape=[batch_size, 1, input_size], dtype=tf.float32))

with tf.variable_scope('decoder') as scope:
    output_cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    decoder_fn_train = tf.contrib.seq2seq.simple_decoder_fn_train(state)
    outputs_train, state_train, context_state_train = tf.contrib.seq2seq.dynamic_rnn_decoder(cell=output_cell,
                                    sequence_length=sequence_length, decoder_fn=decoder_fn_train, inputs=tf_y_oh)

logits = tf.matmul(outputs_train, Why) + by
predictions = tf.argmax(logits, axis=2)

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_y_oh)
learning_rate = 1e-4
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(10001):
        randomStringGenerator = random_string_generator(batch_size=batch_size, chars=sequence_length)
        x, y = next(randomStringGenerator)
#         print('run {0} --------'.format(epoch))

        feed_dict = {tf_x: x, tf_y: y}
        o, l, opt = sess.run([predictions, loss, optimizer], feed_dict=feed_dict)
#         o, l, opt, s = sess.run([predictions, loss, optimizer, outputs_train], feed_dict=feed_dict)

#         print(s.shape)
        if(epoch % 1000 == 0):
            print('run {0} --------'.format(epoch))
#             print('input:', x)
            print('target:', y)
            print('output:', o)
            print('loss:', l)



run 0 --------
target: [[18 17 12  9  4]
 [14  4 11 10  8]
 [ 8 16 16 18 22]
 [18  3  5  7  5]]
output: [[25 25 25 25 25]
 [ 3  3  3  3  3]
 [16 16 16 16 16]
 [22 22 22 22 22]]
loss: [[ 3.99598145  2.03370667  2.95125008  5.04148579  2.95154524]
 [ 4.71193981  1.89380407  4.68060637  3.88002777  4.80885935]
 [ 3.33564949  1.78692639  1.60105419  5.07365799  3.19068766]
 [ 2.77111387  4.94262218  3.98174572  4.44483852  3.89887571]]
run 1000 --------
target: [[ 1  6  4 17 17]
 [ 4 20  3  7  9]
 [19  0 11  9 23]
 [13 18 22  4  8]]
output: [[17 17  4 17 17]
 [ 4  9  3  7  9]
 [19  0 11 11 11]
 [13 18 13  4  8]]
loss: [[ 1.53822148  1.28141224  1.13064313  0.42904699  0.14492948]
 [ 1.76259065  2.72250891  1.27946281  0.965078    1.36483109]
 [ 1.90846515  1.69712126  0.41127491  2.39304447  1.65079737]
 [ 1.47106874  1.28372741  1.98476005  0.9570452   1.31488824]]
run 2000 --------
target: [[14  9  6 24  6]
 [13 11  5  1  5]
 [19 23  3  7  8]
 [25 22 18 12 23]]
output: [[14  9  6 24  6]


In [203]:
print(tf.__version__)

1.0.1


# RNN with MNIST images

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data


In [ ]:
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist = input_data.read_data_sets("MNIST_data/")

In [ ]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(dir(mnist.train))

In [ ]:
def TrainGenerator(batch_size=128):
    while True:
        batch_x, batch_y = mnist.train.next_batch(batch_size*3)
        batch_x = batch_x.reshape(-1, 28, 28)
        
        x = []
        y = []
        for i in range(batch_size):
            img = np.concatenate((batch_x[i], batch_x[i+batch_size], batch_x[i+batch_size*2]), axis=1) #combine 3 images horizontally
            label = [batch_y[i], batch_y[i+batch_size], batch_y[i+batch_size*2]]
            x.append(img)
            y.append(label)
        
        yield np.stack(x), np.stack(y)

gen = TrainGenerator()
x, y = next(gen)

print(x.shape)
print(y.shape)

plt.imshow(x[0], cmap='gray')
print(y[0])